# PyOpenCL

In [14]:
import pyopencl as cl
import pyopencl.array
import pyopencl.clrandom
import numpy as np

In [15]:
ctx = cl.create_some_context()
queue = cl.CommandQueue(ctx)

## Prefix Sum

In [16]:
from pyopencl.scan import GenericScanKernel

`GenericScanKernel(context, dtype, arguments, input_expr, scan_expr, neutral, output_statement)`

In [17]:
x = cl.clrandom.rand(queue, 10**7, dtype=np.float64)

In [28]:
knl = GenericScanKernel(ctx, np.float64,
    arguments="__global double* x",
    input_expr="x[i]*x[i]",
    scan_expr="a+b", neutral="0",
    output_statement="x[i] = item")

In [29]:
knl(x)
print(x)

[  2.39114200e-003   2.91502273e-001   8.92046412e+002 ...,
   2.12036229e+198   2.12036886e+198   2.12037543e+198]
